<a href="https://colab.research.google.com/github/StephaneDucci/Invoice-Converter/blob/main/AnkerProformaConverter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytesseract pillow pdf2image tk

In [4]:
!apt-get update
!apt-get install -y poppler-utils

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [62.5 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,234 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,641 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubun

In [5]:
!apt-get install -y tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 53 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 3s (1,788 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 124595 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
from google.colab import files
import pytesseract
from PIL import Image
from pdf2image import convert_from_path
import os
import pandas as pd
import re

pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

# Funzione per caricare il PDF tramite Colab
def select_pdf_file():
    print("Carica il file PDF")
    uploaded = files.upload()
    for filename in uploaded.keys():
        return filename  # Restituisce il nome del file caricato

# Funzione per estrarre il testo da un'immagine usando pytesseract
def extract_text_from_image(image_path):
    # Imposta il percorso di Tesseract manualmente se non è nel PATH
    img = Image.open(image_path)

    # Configura Tesseract con PSM 4
    custom_config = f'--psm 4'
    text = pytesseract.image_to_string(img, config=custom_config)
    return text

# Funzione per correggere il prezzo totale, eliminando caratteri non necessari
def correggi_totale(prezzo_totale):
    # Rimuovi caratteri come "1)" o altre cifre aggiuntive dopo il prezzo
    prezzo_totale = re.sub(r'[^\d.,]', '', prezzo_totale)  # Rimuove qualsiasi carattere non numerico o virgola/punto

    # Se ci sono più di due cifre decimali (es. "178.808"), tronca a due cifre decimali
    if re.match(r'\d+\.\d{3}', prezzo_totale):  # Se ci sono tre cifre decimali
        prezzo_totale = prezzo_totale[:-1]  # Troncamento per portarlo a due decimali (es. "178.808" -> "178.80")

    return prezzo_totale

def formatta_valori(valore):
    if isinstance(valore, float):
        return f"{valore:.2f}".replace('.', ',')
    return valore

# Funzione per strutturare i dati estratti in colonne
def structure_extracted_data(extracted_text):
    data_processati = []  # Lista per i dati processati
    data_non_processati = []  # Lista per le righe non processate
    data_non_riconosciuti = []  # Lista per le righe con re errata
    processa_dati = False  # Flag per iniziare e fermare il processo di estrazione

    for page_text in extracted_text:
        lines = page_text.split("\n")
        for line in lines:
            line = line.strip()

            # Inizia a processare dopo questa intestazione
            if ('Total (EUR)' in line or
                'Your reference' in line):
                processa_dati = True
                continue  # Salta questa riga, ma imposta il flag per iniziare a processare

            # Smetti di processare se trovi una di queste righe
            if ('Please note that third party payments' in line or
                'Pallet fee' in line or
                'Transport' in line):
                processa_dati = False
                continue  # Salta questa riga e smetti di processare le righe successive

            # Processa solo se il flag è True
            if processa_dati:

                pattern = re.compile(r'([\w\-]+)\s+.*?(\d+[,\.]?\d*)\s+liter\s+(\d+)\s+bottles?\s+(\d+[,\.]\d+)')

                # Esegui il regex per catturare Reference, Volume, Quantity, Price
                match = pattern.match(line)

                try:

                    # Estrarre i gruppi dal regex
                    anker_ref = match.group(1)  # Cattura l'Anker Ref
                    volume = match.group(2).replace(',', '.').strip()  # Converte la virgola in punto
                    quantity = match.group(3).strip()  # Quantità
                    price = float(match.group(4).replace(',', '.'))  # Prezzo per bottiglia
                    total = price * int(quantity)  # Calcola il totale
                    total_str = f"{total:.2f}".replace('.', ',')  # Converte il totale in stringa con virgola come separatore

                    # Determinare la descrizione per differenza
                    end_of_ref = match.end(1)
                    start_of_volume = match.start(2)
                    description = line[end_of_ref:start_of_volume].strip()

                    # Aggiungi i dati processati alla lista
                    data_processati.append([anker_ref, description, volume, quantity, price, total_str])

                except Exception as e:
                    # In caso di errore (ad esempio, una riga spezzata), aggiungi ai non processati
                    data_non_riconosciuti.append(line)
            else:
                data_non_processati.append(line)

    return data_processati, data_non_processati, data_non_riconosciuti

#####################################################################################################################
# INIZIO CODICE MAIN METHOD #########################################################################################
#####################################################################################################################

# 1. Selezionare il PDF con una dialog box
pdf_path = select_pdf_file()

if pdf_path:

    # Lista per memorizzare i percorsi delle immagini generate
    image_paths = []

    # 2. Convertire il PDF in immagini
    images = convert_from_path(pdf_path, dpi=450)

    # 3. Applicare l'OCR su ogni immagine
    extracted_text = []

    for i, image in enumerate(images):

        image_path = f'/content/immagine_{i+1}.png'
        image.save(image_path, 'PNG')

        # Aggiungi il percorso dell'immagine alla lista
        image_paths.append(image_path)

        # Apre l'immagine per usarla con Tesseract
        img = Image.open(image_path)

        # Estraggo il testo dalle immagini usando tesseract
        text = extract_text_from_image(image_path)

        # Aggiunge il testo alla lista creando una stringa unica
        extracted_text.append(text)

        # Chiudi l'immagine per rilasciare le risorse
        img.close()

    # 4. Strutturare i dati
    structured_data, unstructured_data, error_data = structure_extracted_data(extracted_text)

    # 5. Creare un DataFrame per organizzare i dati
    df = pd.DataFrame(structured_data, columns=["Anker Ref.", "Descrizione", "Volume", "Quantity", "Price", "Total"])

    # Applica la formattazione dei valori per Price e Total
    df['Price'] = df['Price'].apply(formatta_valori)
    df['Total'] = df['Total'].apply(formatta_valori)
    df['Volume'] = df['Volume'].apply(lambda v: v.replace('.', ','))  # Sostituisci i punti con virgole nei volumi

    # Salvataggio in Excel e formattazioni
    excel_output_path = '/content/ConvertedInvoice.xlsx'  # Salva l'Excel in Colab
    with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
        df.to_excel(writer, index=False, sheet_name='Invoice')
        workbook = writer.book
        worksheet = writer.sheets['Invoice']

        # Autofit delle colonne
        for column_cells in worksheet.columns:
            max_length = max(len(str(cell.value)) for cell in column_cells)
            column_width = max_length + 2
            worksheet.column_dimensions[column_cells[0].column_letter].width = column_width

        # Centratura e formattazione
        for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=1, max_col=6):
            for cell in row:
                if cell.column_letter in ['A', 'C', 'D']:
                    cell.alignment = Alignment(horizontal='center')
                elif cell.column_letter in ['E', 'F']:
                    cell.alignment = Alignment(horizontal='right')
                elif cell.column_letter == 'B':
                    cell.alignment = Alignment(horizontal='left')

        # Larghezza fissa delle colonne C, D, E, F
        for col in ['C', 'D', 'E', 'F']:
            worksheet.column_dimensions[col].width = 13

        # Blocco e formattazione header
        header_row = worksheet[1]
        for cell in header_row:
            cell.font = Font(bold=True)
        worksheet.freeze_panes = 'A2'

        # Formato valuta per colonne E, F
        for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=5, max_col=6):
            for cell in row:
                cell.value = float(cell.value.replace(',', '.')) if isinstance(cell.value, str) else cell.value
                cell.number_format = '\u20ac#,##0.00'

        # Formato numero con due decimali per colonna C
        for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=3, max_col=3):
            for cell in row:
                cell.value = float(cell.value.replace(',', '.')) if isinstance(cell.value, str) else cell.value
                cell.number_format = '0.00'

        # Aggiunta righe con Transport, Pallet, Other, Total
        next_row = worksheet.max_row + 1
        worksheet[f'E{next_row}'] = 'Transport'
        worksheet[f'E{next_row+1}'] = 'Pallet'
        worksheet[f'E{next_row+2}'] = 'Other'
        worksheet[f'E{next_row+3}'] = 'Total'

        for cell in [worksheet[f'E{next_row}'], worksheet[f'E{next_row+1}'], worksheet[f'E{next_row+2}'], worksheet[f'E{next_row+3}']]:
            cell.font = Font(bold=True)
            cell.alignment = Alignment(horizontal='right')

        # Somma nella colonna F per il totale
        for offset in range(4):
            worksheet[f'F{next_row+offset}'].value = 0
            worksheet[f'F{next_row+offset}'].number_format = '\u20ac#,##0.00'

        worksheet[f'F{next_row+3}'] = f'=SUM(F2:F{next_row+2})'
        total_cell = worksheet[f'F{next_row+3}']
        total_cell.font = Font(bold=True)
        total_cell.alignment = Alignment(horizontal='center')

    print(f"Excel salvato in: {excel_output_path}")
else:
    print("Nessun file PDF selezionato.")


Carica il file PDF


Saving 25NL00002973 - Gen 25.pdf to 25NL00002973 - Gen 25 (1).pdf
Excel salvato in: /content/ConvertedInvoice.xlsx
